Fine tuning BERT for Multi Label Text Classification

Resources:
1. https://huggingface.co/docs/transformers/model_doc/bert
2. https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
3. https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

In [1]:
# -q is for quiet installation
!pip install -q transformers datasets

In [2]:
from datasets import load_dataset
dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")






Found cached dataset sem_eval_2018_task_1 (C:/Users/chopr.000/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 6838
    })
    test: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 3259
    })
    validation: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 886
    })
})

In [4]:
dataset['train'][0]

{'ID': '2017-En-21441',
 'Tweet': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'anger': False,
 'anticipation': True,
 'disgust': False,
 'fear': False,
 'joy': False,
 'love': False,
 'optimism': True,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': True}

In [5]:
labels=[label for label in dataset['train'].features if label not in ['ID','Tweet']]

In [6]:
#ID2Label
#For example
#{2017-En-21441:anticipation,optimism,trust}
for x in dataset['train']:
    ID=x['ID']
    d={}
    lv=[]
    for k,v in x.items():
        if v and k in labels:
            lv.append(k)
            d[ID]=lv       
    print(d)

{'2017-En-21441': ['anticipation', 'optimism', 'trust']}
{'2017-En-31535': ['joy', 'love', 'optimism']}
{'2017-En-21068': ['anger', 'disgust', 'joy', 'optimism']}
{'2017-En-31436': ['joy', 'optimism']}
{'2017-En-22195': ['anger', 'disgust']}
{'2017-En-22190': ['joy']}
{'2017-En-20221': ['anticipation', 'pessimism']}
{'2017-En-22180': ['anger', 'disgust']}
{'2017-En-41344': ['disgust', 'sadness']}
{'2017-En-20759': ['anger', 'disgust']}
{'2017-En-40158': ['disgust', 'pessimism', 'sadness']}
{}
{'2017-En-31337': ['joy', 'love']}
{'2017-En-10902': ['anger', 'sadness']}
{'2017-En-11274': ['anger', 'disgust']}
{'2017-En-11097': ['anger', 'disgust', 'sadness']}
{'2017-En-11475': ['anger', 'disgust', 'fear']}
{'2017-En-10989': ['anger', 'disgust', 'joy', 'sadness']}
{'2017-En-10618': ['joy', 'optimism']}
{'2017-En-40212': ['anger', 'disgust', 'sadness']}
{'2017-En-21011': ['anger', 'disgust']}
{'2017-En-20349': ['fear', 'sadness']}
{'2017-En-40483': ['joy', 'love', 'optimism']}
{'2017-En-3080

{'2017-En-31261': ['joy', 'love', 'optimism']}
{'2017-En-40027': ['pessimism', 'sadness']}
{'2017-En-21308': ['anger', 'disgust']}
{'2017-En-11602': ['anger', 'disgust', 'sadness']}
{'2017-En-30160': ['anticipation', 'joy']}
{'2017-En-41186': ['optimism', 'pessimism']}
{'2017-En-40680': ['joy', 'optimism']}
{'2017-En-30999': ['anticipation', 'joy']}
{'2017-En-20956': ['anger', 'disgust', 'sadness']}
{'2017-En-11080': ['anticipation']}
{'2017-En-31490': ['joy', 'optimism', 'surprise', 'trust']}
{'2017-En-40576': ['joy', 'pessimism']}
{'2017-En-20604': ['fear']}
{'2017-En-10128': ['anger', 'disgust']}
{'2017-En-41424': ['anger', 'disgust', 'sadness']}
{'2017-En-40122': ['sadness']}
{'2017-En-31200': ['anticipation', 'trust']}
{}
{'2017-En-20242': ['anger', 'disgust', 'fear']}
{'2017-En-40562': ['joy', 'pessimism']}
{'2017-En-11668': ['anger', 'sadness']}
{'2017-En-10945': ['anger', 'disgust', 'sadness']}
{'2017-En-10836': ['joy', 'love', 'optimism']}
{'2017-En-30271': ['joy', 'love', 'op

{'2017-En-21386': ['fear', 'optimism']}
{'2017-En-10523': ['anger', 'disgust', 'sadness']}
{'2017-En-11610': ['anger', 'disgust', 'sadness']}
{'2017-En-40203': ['pessimism', 'sadness']}
{'2017-En-40347': ['anger', 'disgust', 'sadness']}
{'2017-En-40324': ['disgust', 'pessimism', 'sadness']}
{'2017-En-11464': ['joy']}
{'2017-En-20958': ['optimism']}
{'2017-En-10513': ['anger', 'disgust']}
{'2017-En-20996': ['fear', 'joy']}
{'2017-En-30070': ['joy', 'optimism']}
{'2017-En-30953': ['joy', 'optimism']}
{'2017-En-20771': ['anger', 'disgust', 'surprise']}
{'2017-En-21431': ['fear', 'pessimism']}
{'2017-En-21687': ['anger', 'disgust', 'optimism', 'sadness']}
{'2017-En-41147': ['disgust']}
{'2017-En-41402': ['optimism', 'sadness']}
{'2017-En-11124': ['disgust', 'joy', 'optimism', 'sadness']}
{'2017-En-21555': ['fear']}
{'2017-En-10615': ['anger', 'disgust', 'sadness']}
{'2017-En-21449': ['anger', 'disgust']}
{'2017-En-40967': ['pessimism', 'sadness']}
{'2017-En-30012': ['joy', 'love', 'optimis

{'2017-En-20557': ['anger', 'disgust']}
{'2017-En-41281': ['fear', 'sadness']}
{'2017-En-20618': ['fear', 'optimism']}
{'2017-En-20640': ['joy', 'optimism']}
{'2017-En-30111': ['anticipation', 'joy', 'love', 'optimism']}
{'2017-En-11186': ['anger', 'joy', 'love', 'trust']}
{'2017-En-11238': ['anger', 'disgust']}
{'2017-En-31268': ['joy', 'surprise']}
{'2017-En-21485': ['anger', 'anticipation', 'disgust']}
{'2017-En-40319': ['disgust', 'sadness']}
{'2017-En-22151': ['optimism', 'trust']}
{'2017-En-30954': ['joy', 'love', 'optimism']}
{'2017-En-11265': ['sadness']}
{'2017-En-31539': ['joy', 'love', 'optimism']}
{'2017-En-21672': ['anger', 'disgust', 'sadness']}
{'2017-En-11509': ['joy']}
{'2017-En-10704': ['joy', 'optimism', 'surprise']}
{'2017-En-20625': ['anger', 'disgust', 'fear']}
{'2017-En-21436': ['joy', 'sadness']}
{'2017-En-30131': ['joy', 'love', 'optimism']}
{'2017-En-31364': ['anticipation', 'trust']}
{'2017-En-30264': ['joy', 'love', 'optimism']}
{'2017-En-20661': ['anger', '

{'2017-En-40734': ['joy', 'optimism', 'trust']}
{'2017-En-31367': ['disgust', 'joy', 'sadness']}
{'2017-En-30947': ['joy', 'love', 'optimism']}
{'2017-En-41419': ['fear', 'sadness']}
{'2017-En-20383': ['anger', 'disgust']}
{'2017-En-11119': ['anger', 'disgust']}
{'2017-En-10097': ['anger', 'disgust', 'sadness']}
{'2017-En-21483': ['fear', 'joy', 'optimism']}
{'2017-En-41078': ['pessimism', 'sadness']}
{'2017-En-31429': ['joy', 'optimism', 'trust']}
{'2017-En-21394': ['anticipation', 'optimism']}
{'2017-En-40216': ['sadness']}
{'2017-En-10104': ['anger', 'disgust']}
{'2017-En-41469': ['disgust']}
{'2017-En-40289': ['anger', 'pessimism', 'sadness']}
{'2017-En-21893': ['anger', 'sadness']}
{'2017-En-20644': ['disgust', 'sadness']}
{'2017-En-10238': ['anger', 'disgust']}
{'2017-En-31408': ['pessimism', 'sadness']}
{'2017-En-21312': ['fear', 'pessimism']}
{'2017-En-31134': ['anticipation', 'joy', 'optimism', 'trust']}
{'2017-En-21865': ['fear', 'pessimism', 'sadness']}
{'2017-En-21976': ['f

...

{'2017-En-10095': ['anger', 'disgust', 'sadness']}
{'2017-En-41292': ['pessimism']}
{'2017-En-41452': ['disgust', 'fear', 'pessimism', 'sadness']}
{'2017-En-21117': ['anticipation', 'joy', 'optimism', 'trust']}
{'2017-En-31064': ['joy', 'love', 'optimism']}
{'2017-En-30847': ['optimism']}
{}
{'2017-En-21753': ['anger', 'pessimism', 'sadness']}
{'2017-En-21608': ['fear', 'joy', 'sadness']}
{'2017-En-41150': ['joy', 'optimism', 'trust']}
{'2017-En-41175': ['joy', 'love']}
{'2017-En-40334': ['anger', 'fear']}
{'2017-En-10079': ['anger', 'disgust']}
{'2017-En-10199': ['anger', 'disgust', 'sadness']}
{}
{'2017-En-10323': ['anger', 'disgust', 'pessimism']}
{'2017-En-20000': ['fear', 'pessimism', 'sadness']}
{'2017-En-21638': ['joy', 'love', 'optimism', 'trust']}
{'2017-En-31297': ['anger', 'disgust']}
{'2017-En-10608': ['anger']}
{'2017-En-22070': ['fear']}
{'2017-En-40611': ['surprise']}
{'2017-En-40570': ['fear', 'joy', 'sadness', 'surprise']}
{'2017-En-22112': ['fear', 'pessimism']}
{'201

{'2017-En-30661': ['anticipation']}
{'2017-En-10768': ['anticipation']}
{'2017-En-30692': ['anticipation', 'joy', 'optimism']}
{'2017-En-31411': ['optimism', 'trust']}
{'2017-En-30514': ['joy', 'optimism']}
{'2017-En-11616': ['anticipation', 'joy', 'optimism', 'surprise']}
{'2017-En-40775': ['anticipation', 'joy', 'optimism']}
{'2017-En-20699': ['anger', 'anticipation', 'disgust', 'surprise']}
{'2017-En-30751': ['joy']}
{'2017-En-31365': ['joy']}
{'2017-En-20743': ['disgust']}
{'2017-En-22014': ['anger', 'disgust', 'fear']}
{'2017-En-30623': ['optimism']}
{}
{'2017-En-30690': ['joy', 'optimism']}
{'2017-En-10908': ['anger', 'disgust']}
{'2017-En-10028': ['anger', 'disgust']}
{'2017-En-10444': ['anger', 'disgust', 'sadness']}
{'2017-En-11148': ['anger', 'disgust', 'pessimism', 'sadness']}
{'2017-En-11641': ['anger', 'disgust']}
{'2017-En-30154': ['joy', 'optimism']}
{'2017-En-41465': ['joy', 'optimism', 'trust']}
{'2017-En-41220': ['anger', 'disgust', 'sadness']}
{'2017-En-31255': ['fea

{'2017-En-10524': ['anger', 'disgust', 'joy']}
{'2017-En-22047': ['disgust', 'fear', 'pessimism', 'sadness']}
{'2017-En-41443': ['fear', 'pessimism', 'sadness']}
{'2017-En-40855': ['anger', 'disgust', 'sadness']}
{'2017-En-40416': ['anticipation', 'disgust', 'fear', 'sadness']}
{'2017-En-20741': ['anger', 'disgust']}
{}
{'2017-En-10276': ['anger', 'sadness']}
{'2017-En-40069': ['pessimism', 'sadness']}
{'2017-En-40038': ['sadness']}
{'2017-En-21369': ['fear']}
{'2017-En-11670': ['anger', 'disgust', 'joy']}
{'2017-En-20429': ['disgust', 'fear']}
{'2017-En-21344': ['disgust']}
{'2017-En-10686': ['anticipation', 'joy', 'optimism']}
{'2017-En-31191': ['anticipation', 'joy']}
{'2017-En-21941': ['anger', 'disgust']}
{'2017-En-40261': ['pessimism', 'sadness']}
{'2017-En-30146': ['joy', 'love', 'optimism']}
{'2017-En-31257': ['anticipation', 'joy']}
{'2017-En-20483': ['anger', 'disgust']}
{'2017-En-40490': ['anger', 'anticipation', 'disgust', 'sadness']}
{'2017-En-31057': ['disgust', 'joy', 's

{}
{'2017-En-41319': ['joy', 'love', 'trust']}
{'2017-En-31454': ['sadness']}
{'2017-En-21380': ['joy', 'optimism']}
{'2017-En-40683': ['joy']}
{'2017-En-41203': ['joy', 'optimism']}
{'2017-En-40481': ['optimism']}
{'2017-En-20355': ['fear', 'optimism']}
{'2017-En-30381': ['joy', 'optimism']}
{'2017-En-11487': ['anger', 'disgust', 'sadness']}
{'2017-En-11207': ['anger', 'disgust', 'fear', 'sadness']}
{'2017-En-10316': ['anger', 'pessimism', 'sadness']}
{'2017-En-10878': ['anger', 'disgust', 'sadness']}
{'2017-En-30335': ['disgust', 'joy', 'optimism']}
{'2017-En-10239': ['anger', 'disgust']}
{'2017-En-41529': ['anger', 'sadness']}
{'2017-En-11149': ['anger', 'disgust']}
{'2017-En-20928': ['anticipation', 'fear', 'joy']}
{'2017-En-20362': ['anger', 'disgust']}
{'2017-En-21197': ['anticipation', 'disgust']}
{'2017-En-11640': ['anger', 'disgust']}
{'2017-En-10210': ['joy', 'surprise']}
{'2017-En-11199': ['anticipation', 'joy', 'love']}
{'2017-En-30213': ['anticipation', 'joy', 'surprise']}

{'2017-En-40678': ['anticipation', 'disgust']}
{'2017-En-21811': ['pessimism', 'sadness']}
{'2017-En-30468': ['joy', 'optimism']}
{'2017-En-41070': ['pessimism', 'sadness']}
{'2017-En-20325': ['anger', 'disgust', 'fear']}
{'2017-En-31151': ['anticipation', 'joy', 'optimism']}
{'2017-En-20718': ['anger', 'disgust']}
{}
{'2017-En-41441': ['anticipation', 'surprise']}
{'2017-En-10886': ['anger', 'disgust', 'pessimism']}
{'2017-En-40662': ['surprise']}
{'2017-En-31003': ['pessimism']}


In [7]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [8]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

from IPython.core.debugger import set_trace

def preprocess_data(examples):
   # set_trace()
    text = examples["Tweet"]
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding["labels"] = labels_matrix.tolist()
    return encoding


The above function has scope of improvement. If all we need is labels for a particular tweet , then there is no need to create
a numpy array for the length of tweet and then condense it down

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Loading cached processed dataset at C:\Users\chopr.000\.cache\huggingface\datasets\sem_eval_2018_task_1\subtask5.english\1.1.0\a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182\cache-8c61f59b4194513e.arrow
Loading cached processed dataset at C:\Users\chopr.000\.cache\huggingface\datasets\sem_eval_2018_task_1\subtask5.english\1.1.0\a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182\cache-2de8dc0fe085cc42.arrow
Loading cached processed dataset at C:\Users\chopr.000\.cache\huggingface\datasets\sem_eval_2018_task_1\subtask5.english\1.1.0\a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182\cache-0bb67575ff30194c.arrow


In [19]:
encoded_dataset.set_format("torch")


{'anger': 0,
 'anticipation': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'love': 5,
 'optimism': 6,
 'pessimism': 7,
 'sadness': 8,
 'surprise': 9,
 'trust': 10}

#### Set up the model for config

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
                                                           "bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id
                                                            )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Set up the parameters for training arguments

In [24]:
batch_size=8
metric_name='f1'

from transformers import Trainer,TrainingArguments

args=TrainingArguments(f"bert-finetuned-sem_eval-english",
                  evaluation_strategy='epoch',
                  metric_for_best_model=metric_name,
                  per_device_train_batch_size=batch_size,
                  num_train_epochs=1,
                  #load_best_model_at_end=True,
                  no_cuda=True,
                  per_device_eval_batch_size=batch_size,
                  weight_decay=0.5,
                  save_strategy='epoch',
                  learning_rate=2e-2,
                  report_to="none" 
                      )



Calculate evaluation metrics


In [25]:
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score
from transformers import EvalPrediction
import torch



def multilabelmetrics(predictions,labels,threshold=0.5):
    sigmoid=torch.nn.Sigmoid()
    probs=sigmoid(torch.Tensor(predictions))
    y_pred=np.zeros(probs.shape)
    y_pred[np.where(probs >threshold)]=1
    y_true=labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

'''
def computemetrics(p: EvalPrediction):
    preds=p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result=multilabelmetrics(predictions=preds,labels=p.label_ids)
    return result

'''
def computemetrics(p):
    preds=p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result=multilabelmetrics(predictions=preds,labels=p.label_ids)
    return result
    
    

In [26]:
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6989, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1179, -0.0810,  0.3685, -0.1769,  0.0785, -0.1107, -0.1827, -0.4042,
          0.5029, -0.0667,  0.7477]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=computemetrics
)

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"
trainer.train()

C:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6838
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 855
  Number of trainable parameters = 109490699
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [87]:
trainer.model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [88]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 886
  Batch size = 8


{'eval_loss': 0.5206711888313293,
 'eval_f1': 0.20676074827699378,
 'eval_roc_auc': 0.5352428454684637,
 'eval_accuracy': 0.012415349887133182,
 'eval_runtime': 273.878,
 'eval_samples_per_second': 3.235,
 'eval_steps_per_second': 0.405,
 'epoch': 1.0}

In [99]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [113]:
logits=outputs.logits
logits

tensor([[ 0.4469, -2.1990, -0.0165, -1.2561, -1.3365, -3.4615, -1.4189, -1.9381,
         -0.8027, -2.8266, -2.8544]], grad_fn=<AddmmBackward0>)

In [136]:
sigmoid=torch.nn.Sigmoid()
#? 
probs=sigmoid(logits.squeeze().cpu())

In [137]:
probs
probs.shape

torch.Size([11])

In [139]:
predictions = np.zeros(probs.shape)

predictions
predictions[np.where(probs >= 0.5)] = 1


predictions


predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['anger']


In [140]:
predictions

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])